In [ ]:
import os
import pandas as pd
from ultralytics import YOLO
import numpy as np
import time

#script genérico, funcionou pra todos menos para o yolov5

# Caminhos
model_path = r'C:\Users\jonas\OneDrive\Área de Trabalho\train_output_yolov11m_100epocas_imgsz1024_adam_sliced_images\weights\best.pt'
results_csv = r'C:\Users\jonas\OneDrive\Área de Trabalho\train_output_yolov11m_100epocas_imgsz1024_adam_sliced_images\results.csv'

# Carregar modelo
model = YOLO(model_path)
metrics = model.val(data=r'C:\Users\jonas\Documents\Doutorado\dota_dataset\yolo-dataset\datasets\yolo-dataset-sliced\dota.yaml')

# Extrair métricas de performance
map50     = metrics.box.map50
map5095   = metrics.box.map
precision = metrics.box.mp
recall    = metrics.box.mr

# Ler CSV para losses
df = pd.read_csv(results_csv)
df.columns = df.columns.str.strip()

cols_train = [c for c in df.columns if c.startswith('train/') and c.endswith('_loss')]
cols_val   = [c for c in df.columns if c.startswith('val/') and c.endswith('_loss')]

df['train_loss'] = df[cols_train].sum(axis=1)
df['val_loss'] = df[cols_val].sum(axis=1)

loss_train_final = df['train_loss'].iloc[-1]
loss_val_final = df['val_loss'].iloc[-1]

# Tamanho do arquivo do modelo
model_size_mb = os.path.getsize(model_path) / 1e6

# Calcular latência média em CPU (inferência com imagem aleatória)
img = np.random.randint(0, 255, (640, 640, 3), dtype=np.uint8)
times = []
for _ in range(20):
    start = time.time()
    _ = model.predict(img, device='cpu')
    times.append(time.time() - start)

latency_ms = (sum(times) / len(times)) * 1000

# Imprimir tabela formatada
print(f'{"Modelo":<10} {"mAP @ 0.50":<12} {"mAP @ 0.50:0.95":<15} Precision  Recall   Latência (CPU)  Loss Final (train)  Loss Final (val)  Arquivo')
print(f'{"yolov11":<10} {map50:<12.3f} {map5095:<15.3f} {precision:<9.3f} {recall:<7.3f} {latency_ms:>10.0f} ms  {loss_train_final:<17.3f} {loss_val_final:<16.3f} {model_size_mb:.1f} MB')
